In [43]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')

model = Sequential()
model.add(MobileNetV2(input_shape=(224, 224, 3), include_top=False))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.layers[0].trainable = False

model.summary()

model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 flatten_3 (Flatten)         (None, 62720)             0         
                                                                 
 dense_6 (Dense)             (None, 128)               8028288   
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 2)                 258       
                                                                 
Total params: 10,286,530
Trainable params: 8,028,546
Non-trainable params: 2,257,984
___________________________________

In [44]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
print(os.getcwd())
print(os.path.exists('da_ta'))

C:\Users\hasib
True


In [45]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'da_ta/with_mask/train',

    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    'da_ta/without_mask/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')


Found 3725 images belonging to 1 classes.
Found 3828 images belonging to 1 classes.


In [46]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [47]:
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=1,
    validation_data=validation_generator,
    validation_steps=len(validation_generator))

model.save('face_mask_detector.h5')


117/117 [==============================] - 402s 3s/step - loss: 886617.6250 - accuracy: 0.5009 - val_loss: 4995207.5000 - val_accuracy: 1.0000


In [51]:
import cv2
import numpy as np

# Load the trained model
model =load_model('face_mask_detector.h5')

# Load the face cascade
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
face_cascade.load('haarcascade_frontalface_default.xml')
# Start the webcam
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(60, 60), flags=cv2.CASCADE_SCALE_IMAGE)

    # Loop over each face
    for (x, y, w, h) in faces:
        # Extract the face ROI
        face = frame[y:y+h, x:x+w]

        # Preprocess the face
        face = cv2.resize(face, (224, 224))
        face = np.expand_dims(face, axis=0)
        face = face / 255.0

        # Make predictions on the face
        pred = model.predict(face)[0]

        # Determine the class label and color
        if pred[0] > pred[1]:
            label = 'With Mask'
            color = (0, 255, 0)  # Green
        else:
            label = 'Without Mask'
            color = (0, 0, 255)  # Red

        # Draw the label and bounding box on the frame
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)

    # Show the frame
    cv2.imshow('Face Mask Detection', frame)

    # Check if the user pressed the 'q' key to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the resources
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 44ms/step


In [ ]:
import cv2
import numpy as np
from keras.models import load_model

model = load_model('face_mask_detector.h5')

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(60, 60), flags=cv2.CASCADE_SCALE_IMAGE)

    for (x, y, w, h) in faces:
        roi = frame[y:y+h, x:x+w]
        roi = cv2.resize(roi, (224, 224))
        roi = np.expand_dims(roi, axis=0)
        roi = roi / 255.0

        pred = model.predict(roi)[0]
        label = 'Mask' if pred[0] > pred[1] else 'No Mask'
        color = (0, 255, 0) if label == 'Mask' else (0, 0, 255)

        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
